In [1]:
# required packages: countdown-timer pyserial numpy av

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import subprocess
import datetime
import multiprocessing as mp
import sys, time
#sys.path.append('simple_pyspin/')
from multicamera_acquisition.simple_pyspin import Camera
import PySpin
from countdown import countdown
import av
import csv
import warnings
from tqdm.autonotebook import tqdm

/tmp/ipykernel_52644/4014358557.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
# from llpyspin import secondary, primary
import numpy as np
import matplotlib.pyplot as plt
import os, time, serial, glob, struct


def packIntAsLong(value):
    """Packs a python 4 byte integer to an arduino long"""
    return struct.pack("i", value)

def get_timing_params(duration, framerate, exposure_time, buffer=50):

    inv_framerate = int(1e6 / framerate)
    num_cycles = int(duration * framerate / 2)

    params = (
        num_cycles,
        exposure_time,
        inv_framerate,
    )

    return params

In [5]:
def wait_for_serial_confirmation(
    expected_confirmation, wait_duration=5, timeout_duration_s=0.1
):
    confirmation = None
    for i in tqdm(
        range(int(recording_duration / timeout_duration_s)),
        desc="Waiting for {} confirmation".format(expected_confirmation),
    ):
        confirmation = arduino.readline().decode("utf-8").strip("\r\n")
        if confirmation == expected_confirmation:
            print("Confirmation recieved: {}".format(confirmation))
            break
        else:
            if len(confirmation) > 0:
                warnings.warn(
                    'PySerial: "{}" confirmation expected, got "{}"". Trying again.'.format(
                        expected_confirmation, confirmation
                    )
                )
    if confirmation != expected_confirmation:
        raise ValueError(
            'Confirmation "{}" signal never recieved from Arduino'.format(
                expected_confirmation
            )
        )
    return confirmation

In [6]:
def count_frames(file_name):
    with av.open(file_name, "r") as reader:
        return reader.streams.video[0].frames


def write_frames(
    filename,
    frames,
    threads=6,
    fps=30,
    crf=10,
    pixel_format="gray8",
    codec="ffv1",
    pipe=None,
    slices=24,
    slicecrc=1,
):

    frame_size = "{0:d}x{1:d}".format(frames.shape[2], frames.shape[1])
    command = [
        "ffmpeg",
        "-y",
        "-loglevel",
        "fatal",
        "-framerate",
        str(fps),
        "-f",
        "rawvideo",
        "-s",
        frame_size,
        "-pix_fmt",
        pixel_format,
        "-i",
        "-",
        "-an",
        "-crf",
        str(crf),
        "-vcodec",
        codec,
        "-preset",
        "ultrafast",
        "-threads",
        str(threads),
        "-slices",
        str(slices),
        "-slicecrc",
        str(slicecrc),
        "-r",
        str(fps),
        filename,
    ]

    if not pipe:
        pipe = subprocess.Popen(command, stdin=subprocess.PIPE, stderr=subprocess.PIPE)
    dtype = np.uint16 if pixel_format.startswith("gray16") else np.uint8
    for i in range(frames.shape[0]):
        pipe.stdin.write(frames[i, :, :].astype(dtype).tobytes())
    return pipe

In [7]:
def get_camera(serial_number=None, exposure_time=2000, gain=15):
    
    cam = Camera(index=str(serial_number))
    cam.init()
    
    cam.GainAuto = 'Off'
    cam.Gain = gain
    cam.ExposureAuto = 'Off'
    cam.ExposureTime = exposure_time
    cam.AcquisitionMode = 'Continuous'
    
    cam.AcquisitionFrameRateEnable = True
    max_fps = cam.get_info('AcquisitionFrameRate')['max']
    cam.AcquisitionFrameRate = max_fps
    
    cam.TriggerMode = 'Off'
    cam.TriggerSource = 'Line3'
    cam.TriggerOverlap = 'ReadOut'
    cam.TriggerSelector = 'FrameStart'
    cam.TriggerActivation = 'RisingEdge'
    cam.TriggerMode = 'On'
    
    return cam

In [8]:
class AcquisitionLoop(mp.Process):
    def __init__(self, write_queue, **camera_params):
        super().__init__()

        self.ready = mp.Event()
        self.primed = mp.Event()
        self.stopped = mp.Event()
        self.write_queue = write_queue
        self.camera_params = camera_params

    def stop(self):
        self.stopped.set()

    def prime(self):
        self.ready.clear()
        self.primed.set()

    def run(self):
        try:
            cam = get_camera(**self.camera_params)
        except:
            print(f'Failed to get camera {self.camera_params["serial_number"]}')

        self.ready.set()
        self.primed.wait()

        cam.start()
        self.ready.set()

        current_frame = 0
        while not self.stopped.is_set():
            try:
                data = cam.get_array(timeout=1000, get_timestamp=True)
                if len(data) != 0:
                    data = data + tuple([current_frame])
                self.write_queue.put(data)
            except PySpin.SpinnakerException:
                pass
            current_frame += 1

        self.write_queue.put(tuple())
        if cam is not None:
            cam.close()

In [9]:
class Writer(mp.Process):
    def __init__(
        self,
        queue,
        video_file_name,
        metadata_file_name,
        camera_serial,
        camera_name,
        **ffmpeg_options
    ):
        super().__init__()
        self.pipe = None
        self.queue = queue
        self.video_file_name = video_file_name
        self.ffmpeg_options = ffmpeg_options
        self.metadata_file_name = metadata_file_name
        self.camera_name = camera_name
        self.camera_serial = camera_serial

        with open(self.metadata_file_name, "w") as metadata_f:
            metadata_writer = csv.writer(metadata_f)
            metadata_writer.writerow(["frame_id", "frame_timestamp", "frame_image_uid"])

    def run(self):
        frame_id = 0
        with open(self.metadata_file_name, "a") as metadata_f:
            metadata_writer = csv.writer(metadata_f)
            while True:
                data = self.queue.get()
                if len(data) == 0:
                    break
                else:
                    # get the computer datetime of the frame
                    frame_image_uid = str(round(time.time(), 5)).zfill(5)
                    img, camera_timestamp, current_frame = data
                    # if the frame is corrupted
                    if img is None:
                        continue
                    metadata_writer.writerow(
                        [
                            current_frame,
                            camera_timestamp,
                            frame_image_uid,
                        ]
                    )
                    self.append(img)
            self.close()

    def append(self, data):
        self.pipe = write_frames(
            self.video_file_name, data[None], pipe=self.pipe, **self.ffmpeg_options
        )

    def close(self):
        if self.pipe is not None:
            self.pipe.stdin.close()

In [10]:
prefix = '../../data/test'

recording_duration = 10
framerate = 30
exposure_time = 2000

params = get_timing_params(recording_duration, framerate, exposure_time)
print(params)

(150, 2000, 33333)


In [11]:
#header = ['frame_camera_serial', 'frame_camera_name', 'frame_id', 'frame_timestamp', 'frame_image_uid']
#write_csv(csv_file, row= header, mode = 'w')
# metadata_writer, metadata_csv_file = write_csv(csv_file, mode='a', return_writer=True)

In [12]:
serial_nums = {
   'top':    22181547,
    'side1':  22181612,
}

In [13]:
# initialize cameras
writers = []
acquisition_loops = []

for k,sn in serial_nums.items():
    
    video_file = f'{prefix}.{k}.{sn}.avi'
    metadata_file = f'{prefix}.{k}.{sn}.metadata.csv'
    
    write_queue = mp.Queue()
    
    writer = Writer(
        write_queue,
        video_file_name = video_file,
        metadata_file_name = metadata_file,
        fps=framerate,
        camera_serial = sn,
        camera_name = k
    )
    
    acquisition_loop = AcquisitionLoop(
        write_queue, 
        serial_number=sn, 
        exposure_time=exposure_time,
        gain=15
    )
    
    writer.start()
    writers.append(writer)
    acquisition_loop.start()
    acquisition_loop.ready.wait()
    acquisition_loops.append(acquisition_loop)
    print(f'Initialized {k}')

Initialized top
Initialized side1


In [14]:
# prepare acquisition loops
for acquisition_loop in acquisition_loops:
    acquisition_loop.prime()
    acquisition_loop.ready.wait()

In [15]:
#`sudo chmod a+rw /dev/ttyACM0`

In [16]:
timeout_duration_s = 0.1

In [17]:
glob.glob('/dev/ttyACM*')

['/dev/ttyACM0']

In [18]:
port = glob.glob('/dev/ttyACM*')[0]
arduino = serial.Serial(port=port, timeout=timeout_duration_s) # timeout is in seconds
#arduino.flushInput()
#arduino.flushOutput()

In [19]:
# delay recording to allow serial connection to connect
time.sleep(2)

In [20]:
# Tell the arduino to start recording by sending along the recording parameters
msg = b"".join(map(packIntAsLong, params))
arduino.write(msg)

# Run acquision
confirmation = wait_for_serial_confirmation("Start")
## TODO: recieve channels flipped
for i in tqdm(range(int(recording_duration / timeout_duration_s))):
    confirmation = arduino.readline().decode("utf-8").strip("\r\n")
    if len(confirmation) > 0:
        print(confirmation)
        # if confirmation[:18] == "new state change: ":
        #    states = confirmation[18:].split(',')[:-1]
        #    frame_num = confirmation[18:].split(',')[-2]
        #    arduino_clock = confirmation[18:].split(',')[-1]

        # TODO: save states to triggerdata.csv

    if confirmation == "Finished":
        break
if confirmation != "Finished":
    confirmation = wait_for_serial_confirmation("Finished")

# end acquisition loops
for acquisition_loop in acquisition_loops:
    acquisition_loop.stop()
    acquisition_loop.join()

# @CALEB: what is the purpose of this?
for writer in writers:
    writer.join()

Waiting for Start confirmation:   0%|          | 0/100 [00:00<?, ?it/s]

Confirmation recieved: Start


  0%|          | 0/100 [00:00<?, ?it/s]

Finished


In [22]:
try:
    print('Frame counts', [count_frames(f'{prefix}.{k}.{sn}.avi') for k, sn in serial_nums.items()])
except:
    print('No Frames')

Frame counts [150, 150]
